In [1]:
import numpy  as np
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"torch version is {torch.__version__} device is {device}")

torch version is 1.2.0 device is cuda



![](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQxqI8ZVhhysQxP955V3hFnPrwdjUcb5LFDzutfQe6GDKjHmMjp)

# Loading the data

In [2]:
with open('data/anna.txt') as f:
    data = f.read()
data[:200]

"Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverything was in confusion in the Oblonskys' house. The wife had\ndiscovered that the husband was carrying on"

# Creating the vocab and tokenizer

In [3]:
vocab = set(data)
itos = dict(enumerate(vocab))
stoi = dict(map(reversed,itos.items()))
encoded = torch.tensor(list(map(stoi.__getitem__ , data)))

In [4]:
def one_hot_encode(arr,n_labels):
    return torch.zeros(*arr.size(),n_labels).scatter_(2,arr.unsqueeze(-1),1)

In [5]:
test_seq = torch.tensor([[3,2,1],[2,3,5],[4,2,5]])
one_hot_encode(test_seq,8)

tensor([[[0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.]],

        [[0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.]]])

In [6]:
def getBatches(arr,batch_size,seq_len):
    
    total_no_char_in_a_batch = batch_size*seq_len 
    n_batches = len(arr)//total_no_char_in_a_batch
    arr = arr[: n_batches * total_no_char_in_a_batch] # trimming extra values
    arr = arr.view(batch_size,-1)
    for n in range(0,arr.shape[1],seq_len):
        x = arr[:,n:n+seq_len]
        y = torch.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_len]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [9]:
batches = getBatches(encoded,32,50)

In [10]:
x, y = next(batches) 
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 tensor([[48, 15, 56, 27, 50, 37, 24, 29, 30, 35],
        [72, 43, 32, 22, 50, 36, 15, 26, 29,  4],
        [79, 26, 29,  4, 15, 43, 35, 56, 63,  4],
        [50,  5, 22, 72, 41, 29,  4, 22, 50, 15],
        [57, 29, 56, 72, 57, 29, 43, 58, 29, 79],
        [59, 56, 22, 57, 26, 29, 79, 50, 80, 27],
        [50, 29, 59, 43, 24, 50, 22, 58, 22, 37],
        [80, 24, 56, 41, 37,  8,  8, 46, 24, 43],
        [63, 20, 26, 29, 15, 37, 64, 79, 29, 41],
        [29, 80, 27, 29, 15, 37, 24, 29, 59, 22]])

y
 tensor([[15, 56, 27, 50, 37, 24, 29, 30, 35, 35],
        [43, 32, 22, 50, 36, 15, 26, 29,  4, 15],
        [26, 29,  4, 15, 43, 35, 56, 63,  4, 56],
        [ 5, 22, 72, 41, 29,  4, 22, 50, 15, 29],
        [29, 56, 72, 57, 29, 43, 58, 29, 79, 43],
        [56, 22, 57, 26, 29, 79, 50, 80, 27, 37],
        [29, 59, 43, 24, 50, 22, 58, 22, 37, 57],
        [24, 56, 41, 37,  8,  8, 46, 24, 43, 72],
        [20, 26, 29, 15, 37, 64, 79, 29, 41, 37],
        [80, 27, 29, 15, 37, 24, 29, 59, 2

# Model Defining

In [11]:
class CharRNN(nn.Module):
    
    def __init__(self, no_tokens, n_hidden=256, n_layers=2,drop_prob=0.5, lr=0.001):
        
        super().__init__()
        
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        
        self.lstm = nn.LSTM(no_tokens, n_hidden, n_layers,  dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden, no_tokens)
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        r_output, hidden = self.lstm(x, hidden)
        
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device))
        return hidden


In [12]:
model = CharRNN(len(vocab)).to(device)
model

CharRNN(
  (lstm): LSTM(83, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=256, out_features=83, bias=True)
)

# Training 

In [15]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
        
    counter = 0
    n_chars = len(vocab)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in getBatches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = x.to(device), y.to(device)

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in getBatches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    inputs, targets = x.to(device), y.to(device)

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [16]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(model, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.3142... Val Loss: 3.2033
Epoch: 1/20... Step: 20... Loss: 3.1744... Val Loss: 3.1298
Epoch: 1/20... Step: 30... Loss: 3.1718... Val Loss: 3.1260
Epoch: 1/20... Step: 40... Loss: 3.1302... Val Loss: 3.1213
Epoch: 1/20... Step: 50... Loss: 3.1562... Val Loss: 3.1195
Epoch: 1/20... Step: 60... Loss: 3.1288... Val Loss: 3.1177
Epoch: 1/20... Step: 70... Loss: 3.1172... Val Loss: 3.1166
Epoch: 1/20... Step: 80... Loss: 3.1328... Val Loss: 3.1151
Epoch: 1/20... Step: 90... Loss: 3.1356... Val Loss: 3.1126
Epoch: 1/20... Step: 100... Loss: 3.1241... Val Loss: 3.1090
Epoch: 1/20... Step: 110... Loss: 3.1150... Val Loss: 3.1011
Epoch: 1/20... Step: 120... Loss: 3.0888... Val Loss: 3.0870
Epoch: 1/20... Step: 130... Loss: 3.0828... Val Loss: 3.0614
Epoch: 2/20... Step: 140... Loss: 3.0544... Val Loss: 3.0167
Epoch: 2/20... Step: 150... Loss: 3.0028... Val Loss: 2.9616
Epoch: 2/20... Step: 160... Loss: 2.9290... Val Loss: 2.8907
Epoch: 2/20... Step: 170... Loss:

Epoch: 10/20... Step: 1350... Loss: 1.7555... Val Loss: 1.7087
Epoch: 10/20... Step: 1360... Loss: 1.7731... Val Loss: 1.7092
Epoch: 10/20... Step: 1370... Loss: 1.7499... Val Loss: 1.7067
Epoch: 10/20... Step: 1380... Loss: 1.7815... Val Loss: 1.6996
Epoch: 10/20... Step: 1390... Loss: 1.7752... Val Loss: 1.7003
Epoch: 11/20... Step: 1400... Loss: 1.7828... Val Loss: 1.6952
Epoch: 11/20... Step: 1410... Loss: 1.7867... Val Loss: 1.6909
Epoch: 11/20... Step: 1420... Loss: 1.7759... Val Loss: 1.6862
Epoch: 11/20... Step: 1430... Loss: 1.7472... Val Loss: 1.6853
Epoch: 11/20... Step: 1440... Loss: 1.7800... Val Loss: 1.6826
Epoch: 11/20... Step: 1450... Loss: 1.7105... Val Loss: 1.6791
Epoch: 11/20... Step: 1460... Loss: 1.7300... Val Loss: 1.6845
Epoch: 11/20... Step: 1470... Loss: 1.7124... Val Loss: 1.6749
Epoch: 11/20... Step: 1480... Loss: 1.7366... Val Loss: 1.6742
Epoch: 11/20... Step: 1490... Loss: 1.7342... Val Loss: 1.6696
Epoch: 11/20... Step: 1500... Loss: 1.7176... Val Loss:

Epoch: 20/20... Step: 2660... Loss: 1.5198... Val Loss: 1.4772
Epoch: 20/20... Step: 2670... Loss: 1.5379... Val Loss: 1.4756
Epoch: 20/20... Step: 2680... Loss: 1.5246... Val Loss: 1.4785
Epoch: 20/20... Step: 2690... Loss: 1.5048... Val Loss: 1.4751
Epoch: 20/20... Step: 2700... Loss: 1.5236... Val Loss: 1.4758
Epoch: 20/20... Step: 2710... Loss: 1.4891... Val Loss: 1.4726
Epoch: 20/20... Step: 2720... Loss: 1.5030... Val Loss: 1.4736
Epoch: 20/20... Step: 2730... Loss: 1.4899... Val Loss: 1.4712
Epoch: 20/20... Step: 2740... Loss: 1.4856... Val Loss: 1.4688
Epoch: 20/20... Step: 2750... Loss: 1.4902... Val Loss: 1.4729
Epoch: 20/20... Step: 2760... Loss: 1.4877... Val Loss: 1.4707
Epoch: 20/20... Step: 2770... Loss: 1.5290... Val Loss: 1.4668
Epoch: 20/20... Step: 2780... Loss: 1.5281... Val Loss: 1.4681


# Sample Prediction

In [47]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = torch.tensor([[stoi[char]]])
        x = one_hot_encode(x, len(vocab))
        inputs = x.to(device)
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(vocab))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return itos[char], h

In [48]:
def sample(net, size, prime='The', top_k=None):
        
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [61]:
print(sample(model, 2000, top_k=5, prime="Anna "))

Anna was not hand what had not been as as and the
preame to hard all
the stept of the posest, she was as him, and was solieters with a still, to
say that
he had a mind the man, to have tellend a change what she went that
the care and the perisitaning and that, and
hose of sores and a taste,
she could not speak to her times to be to himself with
thore, was
at him of that walking his horse of the convilial of the same words with her and there was nility.

"You were betwien at you was once," she
taked husbouss at his hand.

"Would to see them as that is a point of the most
answilion, and
have, tearing her the sord
of the second," see and she
canded to the child be beginning and she worded himself, with something to her head was that the since was brought
her shear of him and and a same than were have to take to staid of the seement, tread of a pression of which she could not head shand, standing to the chirch and that he was bread, as he was bound than it, and was to be all a sense time o